In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
pd.set_option("display.max_columns", None)

In [23]:
stats = pd.read_csv("/home/anuroopa/Desktop/Data_analysis_projects/Pokemon/stats.csv", index_col = 0)
type_chart = pd.read_csv("/home/anuroopa/Desktop/Data_analysis_projects/Pokemon/Pokemon_Type_Chart.csv", index_col = 0)
details = pd.read_csv("/home/anuroopa/Desktop/Data_analysis_projects/Pokemon/data.csv", index_col = 0)
details = stats.merge(details)

In [4]:
type_chart

,Normal,Fire,Water,Electric,Grass,Ice,Fighting,Poison,Ground,Flying,Psychic,Bug,Rock,Ghost,Dragon,Dark,Steel,Fairy,Nil
Normal,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,0.5,1.0,1.0
Fire,1.0,0.5,0.5,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,0.5,1.0,2.0,1.0,1.0
Water,1.0,2.0,0.5,1.0,0.5,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0,1.0,1.0
Electric,1.0,1.0,2.0,0.5,0.5,1.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,1.0
Grass,1.0,0.5,2.0,1.0,0.5,1.0,1.0,0.5,2.0,0.5,1.0,0.5,2.0,1.0,0.5,1.0,0.5,1.0,1.0
Ice,1.0,0.5,0.5,1.0,2.0,0.5,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,1.0
Fighting,2.0,1.0,1.0,1.0,1.0,2.0,1.0,0.5,1.0,0.5,0.5,0.5,2.0,0.0,1.0,2.0,2.0,0.5,1.0
Poison,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,1.0,1.0,1.0,0.5,0.5,1.0,1.0,0.0,2.0,1.0
Ground,1.0,2.0,1.0,2.0,0.5,1.0,1.0,2.0,1.0,0.0,1.0,0.5,2.0,1.0,1.0,1.0,2.0,1.0,1.0
Flying,1.0,1.0,1.0,0.5,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,0.5,1.0,1.0,1.0,0.5,1.0,1.0


In [26]:
def preparing_data(details, level=50):
    details = details.copy()
    stats = ["attack", "special_attack", "defense", "special_defense", "speed", "hp"]
    for stat in stats:
        base = details[stat]
        details[stat] = np.floor(((2 * base + 31) * level) / 100) + (level + 10 if stat == "hp" else 5)
    details[["type1", "type2"]] == details["type"].str.split(" / ", expand=True)
    details["type2"] = details["type2"].fillna("Nil")
    details["main_attack"] = np.maximum(details["attack"], details["special_attack"])
    return details

In [11]:
def stimulator(details, type_chart, name):
    """
    Parameters: 
    details - pd.DataFrame containg pokemon details scraped from bulbapedia
    type_chart - pd.DataFrame of pokemon damage type chart with attacking type 
    as index and defensing type as column names
    name - str of name of the particular pokemon you want to stimulate
    Returns:
    pd.DataFrame with columns name, outcome
    """
    
    details = details.copy()
    
    level = 50 
    
    details["hp"] = np.floor(((2 * details["hp"] + 31) * level) / 100) + level + 10
    details["attack"] = np.floor(((2 * details["attack"] + 31) * level) / 100) + 5
    details["defense"] = np.floor(((2 * details["defense"] + 31) * level) / 100) + 5
    details["special_attack"] = np.floor(((2 * details["special_attack"] + 31) * level) / 100) + 5
    details["special_defense"] = np.floor(((2 * details["special_defense"] + 31) * level) / 100) + 5
    details["speed"] = np.floor(((2 * details["speed"] + 31) * level) / 100) + 5
    details[["type1", "type2"]] = details["type"].str.split(" / ", expand = True)
    details["type2"] = details["type2"].fillna("Nil")
    
    details["main_attack"] = np.maximum(details["attack"], details["special_attack"])
    
    pokemon = details[details["name"].str.contains(name, case = False)]
    if len(pokemon) < 1:
        print(f"The given name: {name} is not a name of available pokemon")
        return None

    attack = pokemon["attack"].iloc[0]
    special_attack = pokemon["special_attack"].iloc[0]
    details["main_defense"] = np.where(attack >= special_attack, details["defense"], details["special_defense"])

    speed = pokemon["speed"].iloc[0]
    defense = pokemon["defense"].iloc[0]
    special_defense = pokemon["special_defense"].iloc[0]
    main_attack = pokemon["main_attack"].iloc[0]
    type1 = pokemon["type1"].iloc[0]
    type2 = pokemon["type2"].iloc[0]
    hp = pokemon["hp"].iloc[0]
    details["pokemon_defense"] = np.where(details["attack"] >= details["special_attack"],defense, special_defense)

        
    details["A/D_pokemon"] = main_attack / details["main_defense"]
    details["A/D_others"] = details["main_attack"] / details["pokemon_defense"]
    
    type11_offense = np.array([type_chart.loc[type1, :][x] for x in details["type1"]])
    type12_offense = np.array([type_chart.loc[type1, :][x] for x in details["type2"]])
    details["pokemon_type1_offense"] = type11_offense * type12_offense

    if type2 != "Nil":
        type21_offense = np.array([type_chart.loc[type2, :][x] for x in details["type1"]])
        type22_offense = np.array([type_chart.loc[type2, :][x] for x in details["type2"]])  
        details["pokemon_type2_offense"] = type21_offense * type22_offense
        details["pokemon_type_offense"] = np.maximum(details["pokemon_type1_offense"], 
                                                     details["pokemon_type2_offense"])
    else:
        details["pokemon_type_offense"] = details["pokemon_type1_offense"]

    type11_other_offense = np.array([type_chart.loc[x, :][type1] for x in details["type1"]])
    type12_other_offense = np.array([type_chart.loc[x, :][type1] for x in details["type2"]])
    details["others_type1_offense"] = type11_other_offense * type12_other_offense

    type21_other_offense = np.array([type_chart.loc[x, :][type2] for x in details["type1"]])
    type22_other_offense = np.array([type_chart.loc[x, :][type2] for x in details["type2"]])
    details["others_type2_offense"] = type21_other_offense * type22_other_offense

    if type2 != "Nil":
        details["others_type_offense"] = np.maximum(details["others_type1_offense"],
                                                    details["others_type2_offense"])
    else:
        details["others_type_offense"] = np.maximum(details["others_type1_offense"], 
                                                    details["others_type2_offense"])
    
    power = 80
    
    details["damage_dealt_pokemon"] = (((2*level/5+2)*power*details["A/D_pokemon"])/50+2)*details["pokemon_type_offense"]
    details["damage_dealt_others"] = (((2*level/5+2)*power*details["A/D_others"])/50+2)*details["others_type_offense"]

    details["chance"] = np.where(details["speed"] < speed, 1, 
                                 np.where(details["speed"] > speed, 2,
                                 np.random.randint(1, 3, len(details))))
    
    details["turns_to_win_pokemon"] = np.ceil(details["hp"] / details["damage_dealt_pokemon"])
    details["turns_to_win_others"] = np.ceil(hp / details["damage_dealt_others"])
    
    details["outcome"] = np.where(details["turns_to_win_pokemon"] < details["turns_to_win_others"], "win",
                                  np.where(details["turns_to_win_pokemon"] > details["turns_to_win_others"], "lose",
                                           np.where(details["chance"] == 1, "win", "lose")))

    return details.loc[:, ["name", "outcome", "turns_to_win_others", "turns_to_win_pokemon"]]

In [14]:
s = stimulator(details, type_chart, "Palkia")

In [15]:
s[s["name"] == "Dialga"]

,name,outcome,turns_to_win_others,turns_to_win_pokemon
482,Dialga,lose,2.0,4.0


In [38]:
def count_results(tupe, details, type_chart):
    """
    Parameters: tupe - Tuple with names of pokemon
    Return a dataframe with wins and loses
    """
    results = []
    for i, name in enumerate(tupe):
        result = stimulator(details, type_chart, name)
        #print(result)
        outcome = {"name": name, "win": (result["outcome"] == "win").sum(), "lose": (result["outcome"] == "lose").sum()}
        results.append(outcome)
    return pd.DataFrame(results)

In [9]:
tupe = tuple(details["name"])
answers = count_results(tupe, details, type_chart)

In [12]:
answers.to_csv("/home/anuroopa/Desktop/Data_analysis_projects/Pokemon/result.csv")

In [18]:
def one_vs_one(details, type_chart, for_, against):
    """
    Parameters: details - pd.DataFrame() containing necessary details of Pokemon scraped from Bulbapedia
    type_chart: pd.DataFrame() containing Pokemon type chart with offensive typing as row index and defensive a column headings
    for_: str containing the name of the Pokemon you choose
    against: str containing the name of the Pokemon you want to play against
    return: a str containing "win", "lose" or "draw"
    """
    details = details.copy()
    
    level = 50 
    
    details["hp"] = np.floor(((2 * details["hp"] + 31) * level) / 100) + level + 10
    details["attack"] = np.floor(((2 * details["attack"] + 31) * level) / 100) + 5
    details["defense"] = np.floor(((2 * details["defense"] + 31) * level) / 100) + 5
    details["special_attack"] = np.floor(((2 * details["special_attack"] + 31) * level) / 100) + 5
    details["special_defense"] = np.floor(((2 * details["special_defense"] + 31) * level) / 100) + 5
    details["speed"] = np.floor(((2 * details["speed"] + 31) * level) / 100) + 5
    details[["type1", "type2"]] = details["type"].str.split(" / ", expand = True)
    details["type2"] = details["type2"].fillna("Nil")
    
    details["main_attack"] = np.maximum(details["attack"], details["special_attack"])
    for_ = for_.capitalize()
    against = against.capitalize()
    for_data = details[details["name"] == for_]
    against_data = details[details["name"] == against]

    if len(for_data) < 1 or len(against_data) < 1:
        print("Enter Valid Pokemon Names")
        return None

    for_attack = for_data["main_attack"].iloc[0]
    for_defense = np.where(against_data["attack"] >= against_data["special_attack"],
                          for_data["defense"].iloc[0], for_data["special_defense"].iloc[0])
    for_speed = for_data["speed"].iloc[0]
    for_hp = for_data["hp"].iloc[0]
    for_type1 = for_data["type1"].iloc[0]
    for_type2 = for_data["type2"].iloc[0]
    against_attack = against_data["main_attack"].iloc[0]
    against_defense = np.where(for_data["attack"] >= for_data["special_attack"],
                              against_data["defense"].iloc[0], against_data["special_defense"].iloc[0])
    against_speed = against_data["speed"].iloc[0]
    against_hp = against_data["hp"].iloc[0]
    against_type1 = against_data["type1"].iloc[0]
    against_type2 = against_data["type2"].iloc[0]

    for_ad = for_attack/against_defense
    against_ad = against_attack/for_defense

    for_offense_type11 = type_chart.loc[for_type1, :][against_type1]
    for_offense_type12 = type_chart.loc[for_type1, :][against_type2]
    for_offense_only_type1 = for_offense_type11 * for_offense_type12
    for_offense_type21 = type_chart.loc[for_type2, :][against_type1]
    for_offense_type22 = type_chart.loc[for_type2, :][against_type2]
    for_offense_only_type2 = for_offense_type21 * for_offense_type22
    for_offense = np.maximum(for_offense_only_type1, for_offense_only_type2)

    against_offense_type11 = type_chart.loc[against_type1, :][for_type1]
    against_offense_type12 = type_chart.loc[against_type1, :][for_type2]
    against_offense_only_type1 = against_offense_type11 * against_offense_type12
    against_offense_type21 = type_chart.loc[against_type2, :][for_type1]
    against_offense_type22 = type_chart.loc[against_type2, :][for_type2]
    against_offense_only_type2 = against_offense_type21 * against_offense_type22
    against_offense = np.maximum(against_offense_only_type1, against_offense_only_type2)

    power = 80
    
    damage_dealt_for = (((2*level/5+2)*power*for_ad)/50+2)*for_offense
    damage_dealt_against = (((2*level/5+2)*power*against_ad)/50+2)*against_offense
    
    turns_taken_for = np.ceil(against_hp/damage_dealt_for)
    turns_taken_against = np.ceil(for_hp/damage_dealt_against)
    if turns_taken_for[0] < turns_taken_against[0]:
        return pd.DataFrame({"outcome": "won",
                            "turns_taken_for": turns_taken_for, "turns_taken_against": turns_taken_against})
    elif turns_taken_for[0] == turns_taken_against[0]:
        if for_speed > against_speed:
            return pd.DataFrame({"outcome": "won",
                            "turns_taken_for": turns_taken_for, "turns_taken_against": turns_taken_against})
        elif for_speed == against_speed:
            return pd.DataFrame({"outcome": "draw",
                            "turns_taken_for": turns_taken_for, "turns_taken_against": turns_taken_against})
        else:
            return pd.DataFrame({"outcome": "lose",
                            "turns_taken_for": turns_taken_for, "turns_taken_against": turns_taken_against})
    else:
        return pd.DataFrame({"outcome": "lose",
                            "turns_taken_for": turns_taken_for, "turns_taken_against": turns_taken_against})
        

In [22]:
def stimulator(details, type_chart, name):
    """
    Parameters: 
    details - pd.DataFrame containg pokemon details scraped from bulbapedia
    type_chart - pd.DataFrame of pokemon damage type chart with attacking type 
    as index and defensing type as column names
    name - str of name of the particular pokemon you want to stimulate
    Returns:
    pd.DataFrame with columns name, outcome
    """
    
    details = details.copy()
    
    level = 50 
    
    pokemon = details[details["name"].str.contains(name, case = False)]
    if len(pokemon) < 1:
        print(f"The given name: {name} is not a name of available pokemon")
        return None

    attack = pokemon["attack"].iloc[0]
    special_attack = pokemon["special_attack"].iloc[0]
    details["main_defense"] = np.where(attack >= special_attack, details["defense"], details["special_defense"])
    number = pokemon["number"].astype(str).str.zfill(3).iloc[0]
    speed = pokemon["speed"].iloc[0]
    defense = pokemon["defense"].iloc[0]
    special_defense = pokemon["special_defense"].iloc[0]
    main_attack = pokemon["main_attack"].iloc[0]
    type1 = pokemon["type1"].iloc[0]
    type2 = pokemon["type2"].iloc[0]
    hp = pokemon["hp"].iloc[0]
    details["pokemon_defense"] = np.where(details["attack"] >= details["special_attack"],defense, special_defense)

        
    details["A/D_pokemon"] = main_attack / details["main_defense"]
    details["A/D_others"] = details["main_attack"] / details["pokemon_defense"]
    
    type11_offense = np.array([type_chart.loc[type1, :][x] for x in details["type1"]])
    type12_offense = np.array([type_chart.loc[type1, :][x] for x in details["type2"]])
    details["pokemon_type1_offense"] = type11_offense * type12_offense

    if type2 != "Nil":
        type21_offense = np.array([type_chart.loc[type2, :][x] for x in details["type1"]])
        type22_offense = np.array([type_chart.loc[type2, :][x] for x in details["type2"]])  
        details["pokemon_type2_offense"] = type21_offense * type22_offense
        details["pokemon_type_offense"] = np.maximum(details["pokemon_type1_offense"], 
                                                     details["pokemon_type2_offense"])
    else:
        details["pokemon_type_offense"] = details["pokemon_type1_offense"]

    type11_other_offense = np.array([type_chart.loc[x, :][type1] for x in details["type1"]])
    type12_other_offense = np.array([type_chart.loc[x, :][type1] for x in details["type2"]])
    details["others_type1_offense"] = type11_other_offense * type12_other_offense

    type21_other_offense = np.array([type_chart.loc[x, :][type2] for x in details["type1"]])
    type22_other_offense = np.array([type_chart.loc[x, :][type2] for x in details["type2"]])
    details["others_type2_offense"] = type21_other_offense * type22_other_offense

    if type2 != "Nil":
        details["others_type_offense"] = np.maximum(details["others_type1_offense"],
                                                    details["others_type2_offense"])
    else:
        details["others_type_offense"] = np.maximum(details["others_type1_offense"], 
                                                    details["others_type2_offense"])
    
    power = 80
    
    details["damage_dealt_pokemon"] = (((2*level/5+2)*power*details["A/D_pokemon"])/50+2)*details["pokemon_type_offense"]
    details["damage_dealt_others"] = (((2*level/5+2)*power*details["A/D_others"])/50+2)*details["others_type_offense"]

    details["chance"] = np.where(details["speed"] < speed, 1, 
                                 np.where(details["speed"] > speed, 2,
                                 np.random.randint(1, 3, len(details))))
    
    details["turns_to_win_pokemon"] = np.ceil(details["hp"] / details["damage_dealt_pokemon"])
    details["turns_to_win_others"] = np.ceil(hp / details["damage_dealt_others"])
    
    details["outcome"] = np.where(details["turns_to_win_pokemon"] < details["turns_to_win_others"], "win",
                                  np.where(details["turns_to_win_pokemon"] > details["turns_to_win_others"], "lose",
                                           np.where(details["chance"] == 1, "win", "lose")))

    return details.loc[:, ["name", "outcome"]], number

In [2]:
details = pd.read_csv("data_for_stimulation.csv", index_col=0)
type_chart = pd.read_csv("Pokemon_Type_Chart.csv", index_col=0)

In [ ]:
def stimulation_basedon_moves(details, type_chart, for_, against):
    details.copy()
    

In [59]:
data = requests.get("https://www.serebii.net/pokedex-sv/charizard/").text

In [60]:
soup = BeautifulSoup(data, "lxml")

In [61]:
foo = soup.find_all("td", class_="fooinfo")

In [65]:
def scrap_pokemon_moves(pokemon_name):
    pokemon = pokemon_name.replace(" ", "").lower()
    try:
        response = requests.get(f"https://www.serebii.net/pokedex-sv/{pokemon}/")
        response.raise_for_status
        text = response.text
    except requests.exceptions.RequestException as e:
        print(f"Request error {e} occured")
        return None
    soup = BeautifulSoup(text, "lxml")
    # get moves
    moves = []
    moves_data = soup.find_all("td", class_="fooinfo")
    for data in moves_data:
        if data.find("a") != None:
            link = data.find("a").get("href")
            if link != None:
                if "attackdex" in link:
                    move_data = data.find("a").text
                    if "TM" in move_data:
                        None
                    else:
                        moves.append(move_data)
    return {"name": pokemon_name, "moves": moves}

In [62]:
moves = []
for i, f in enumerate(foo):
    if f.find("a") != None:
        link = f.find("a").get("href")
        if link != None:
            if "attackdex" in link:
                move_data = f.find("a").text
                if "TM" in move_data:
                    None
                else:
                    moves.append(move_data)
moves

['Fire Blast',
 'Air Slash',
 'Tackle',
 'Growl',
 'Dragon Claw',
 'Ember',
 'Smokescreen',
 'Dragon Breath',
 'Fire Fang',
 'Fire Spin',
 'Slash',
 'Flamethrower',
 'Dragon Rush',
 'Flare Blitz',
 'Dragon Claw',
 'Roar',
 'Brick Break',
 'Rock Slide',
 'Fire Fang',
 'Protect',
 'Thunder Punch',
 'Crunch',
 'Swift',
 'Dig',
 'Fire Punch',
 'Swords Dance',
 'Body Slam',
 'Endure',
 'Rock Tomb',
 'Fire Blast',
 'Fly',
 'Hyper Beam',
 'Overheat',
 'Giga Impact',
 'Double-Edge',
 'Will-O-Wisp',
 'Shadow Claw',
 'Flamethrower',
 'Solar Beam',
 'Heat Wave',
 'Earthquake',
 'Fire Spin',
 'Dragon Pulse',
 'Hurricane',
 'Substitute',
 'Iron Tail',
 'Outrage',
 'Focus Blast',
 'Flare Blitz',
 'Solar Beam ',
 'Air Slash',
 'Scratch',
 'Growl',
 'Ember',
 'Smokescreen',
 'Heat Wave',
 'Dragon Claw',
 'Dragon Breath',
 'Fire Fang',
 'Slash',
 'Flamethrower',
 'Scary Face',
 'Fire Spin',
 'Inferno',
 'Flare Blitz',
 'Take Down',
 'Scary Face',
 'Protect',
 'Fire Fang',
 'Acrobatics',
 'Fire Spin',
 

In [35]:
pd.DataFrame({"name": "Greninja", "moves": (moves)}, columns = ["name", "moves"])

,name,moves
0,Greninja,Fire Blast
1,Greninja,Air Slash
2,Greninja,Tackle
3,Greninja,Growl
4,Greninja,Dragon Claw
...,...,...
129,Greninja,Iron Tail
130,Greninja,Metal Claw
131,Greninja,Ancient Power
132,Greninja,Dragon Rush
